In [1]:
import praw
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
user_agent="Scrapper - 1 by /u/Ordinary-Ticket-4825"
reddit=praw.Reddit(
    client_id="YqhNjtynahlkYmE96uPlLw",
    client_secret="E8FPo7Kg4UOwYYh-iczTjUESX4Uc7w",
user_agent=user_agent
)                   

In [3]:
# Define subreddits and keywords
subreddits = ["india", "finance", "economy", "business","IndiaFinance","IndiaSpeaks"]
keywords = [ "Union Budget 2025", "India Budget 2025", "Budget 2025", "Modi Budget 2025"]

# List to store comments
comments_list = []

# Search each subreddit for each keyword
for subreddit in subreddits:
    for query in keywords:
        sub = reddit.subreddit(subreddit)
        for post in sub.search(query, limit=100):  # Fetch top 5 posts per query
            post.comments.replace_more(limit=0)  # Avoid "More Comments" placeholders
            for comment in post.comments.list():
                comments_list.append([subreddit, query, comment.body])

# Convert data to DataFrame
df = pd.DataFrame(comments_list, columns=["Subreddit", "Keyword", "Comment"])
print(df.head())

  Subreddit            Keyword  \
0     india  Union Budget 2025   
1     india  Union Budget 2025   
2     india  Union Budget 2025   
3     india  Union Budget 2025   
4     india  Union Budget 2025   

                                             Comment  
0  To get a realistic picture, this should be wei...  
1  https://preview.redd.it/k3bh84byhrge1.jpeg?wid...  
2                 Per capita numbers anyone, please?  
3  What many people also do not understand is tha...  
4  What are they doing in sikkim with 5k crores? ...  


In [4]:
df.shape

(35273, 3)

In [9]:
df.Keyword.value_counts()

Keyword
India Budget 2025    12308
Union Budget 2025    11345
Modi Budget 2025      9608
Budget 2025           2012
Name: count, dtype: int64

In [10]:
df.Subreddit.value_counts()

Subreddit
india           16401
IndiaSpeaks     13599
economy          4850
business          365
IndiaFinance       30
finance            28
Name: count, dtype: int64

In [11]:
df.to_excel("reddit_sentiment_analysis_2.xlsx", index=False, engine="openpyxl")
print("Data exported to reddit_sentiment_analysis_2.xlsx successfully!")

Data exported to reddit_sentiment_analysis_2.xlsx successfully!


In [ ]:
import re
import nltk
from nltk.corpus import stopwords

nltk.download("stopwords")
stop_words = set(stopwords.words("english"))

def clean_text(text):
    text = re.sub(r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE)  # Remove URLs
    text = re.sub(r"[^A-Za-z0-9 ]+", "", text)  # Remove special characters
    text = " ".join([word.lower() for word in text.split() if word.lower() not in stop_words])  # Remove stopwords
    return text

df["Cleaned_Comment"] = df["Comment"].apply(clean_text)
print(df.head())


In [ ]:
from textblob import TextBlob

def get_sentiment(text):
    analysis = TextBlob(text)
    if analysis.sentiment.polarity > 0:
        return "Positive"
    elif analysis.sentiment.polarity < 0:
        return "Negative"
    else:
        return "Neutral"

df["Sentiment"] = df["Cleaned_Comment"].apply(get_sentiment)
print(df["Sentiment"].value_counts())


In [ ]:
import matplotlib.pyplot as plt

df["Sentiment"].value_counts().plot(kind="pie", autopct="%1.1f%%", colors=["green", "red", "blue"])
plt.title("Sentiment Analysis of Union Budget 2025 Reddit Comments")
plt.show()

In [ ]:
from wordcloud import WordCloud

text = " ".join(comment for comment in df["Cleaned_Comment"])
wordcloud = WordCloud(width=800, height=400, background_color="white").generate(text)

plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title("Word Cloud of Union Budget 2025 Reddit Comments")
plt.show()


In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer

nltk.download("vader_lexicon")
sia = SentimentIntensityAnalyzer()

def get_vader_sentiment(text):
    score = sia.polarity_scores(text)["compound"]
    return "Positive" if score > 0 else "Negative" if score < 0 else "Neutral"

df["VADER_Sentiment"] = df["Cleaned_Comment"].apply(get_vader_sentiment)
print(df["VADER_Sentiment"].value_counts())
